# CLRS 20.5-5

## Exercise
Compute the `component graph` of directed graph G = (V, E). Make sure there is at most one edge between two vertices in the `component graph`.

## Answer

- Run DFS on init graph to calculate finish times
- Traverse graph to form GT graph
- Run another DFS on GT with G finish times order
    - Each forest tree is Strongly Connected Component
- Create new `component graph` CG
    - create component graph vertice by traversing all SCCs
    - create map of original -> CG vertices, like {'a'-> 'abc', 'b' -> 'abc', ...}
    - Traverse all edges:
        - if both head and tail belong to the same SCC, do nothing
        - if head and tail belong to different SCC, create new distinct edges between component graph vertices

In [46]:
from collections import defaultdict, deque

In [3]:
g = {}
g['a'] = ['b']
g['b'] = ['c']
g['c'] = ['a']

g['d'] = ['b', 'b', 'e']

for u, vs in g.items():
    print(f"{u} : {vs}")

a : ['b']
b : ['c']
c : ['a']
d : ['b', 'b', 'e']


## Get DFS Finish Times

In [44]:
def get_finish_times(g):
    '''
    Returns vertices ordered in finished DFS times in desc order.
    '''
    m = {}
    t = 0
    
    seen = set()
    for u in g.keys():
        if u in seen:
            continue
        
        st = deque([u])
        t += 1
        seen.add(u)
        while st:
            u = st[-1]
            
            v = None
            if u in g:
                for x in g[u]:
                    if x in seen:
                        continue
                    v = x
                    break
                
            if v is None:
                t += 1
                m[u] = t
                seen.add(u)
                st.pop()
            else:
                t += 1
                seen.add(v)
                st.append(v)
                
    
    return dict(sorted(m.items(), key=lambda x: x[1], reverse=True))

In [45]:
finish_times = get_finish_times(g)
finish_times

{'d': 10, 'e': 9, 'a': 6, 'b': 5, 'c': 4}

In [68]:
assert finish_times == {'d': 10, 'e': 9, 'a': 6, 'b': 5, 'c': 4}

## Traverse Graph

In [31]:
def traverse(g):
    '''
    Return a bigraph with traversed edges.
    '''
    gt = defaultdict(list)
    for u, vs in g.items():
        for v in vs:
            gt[v].append(u)
    return gt

In [32]:
gt = traverse(g)

for u, vs in gt.items():
    print(f"{u} : {vs}")

b : ['a', 'd', 'd']
c : ['b']
a : ['c']
e : ['d']


## Traverse GT in descending finish times order

In [57]:
def scc(gt, finish_times):
    sccs = []
    seen = set()

    def dfs_visit(u, scc):
#         print(u)
        seen.add(u)
        scc.append(u)
        
        for v in gt[u]:
            if v in seen:
                continue
            dfs_visit(v, scc)
    
    for u in finish_times.keys():
        if u in seen:
            continue
        scc = []
        sccs.append(scc)
        dfs_visit(u, scc)
        
    return sccs


In [60]:
sccs = scc(gt, finish_times)
assert sccs  == [['d'], ['e'], ['a', 'c', 'b']]

In [63]:
def to_component_graph(g, sccs):
    '''
    Returns new Component Graph each vertex is a list of SCC vertices and there is at most one edge between two vertices of component graph.
    For example,
    input: 
        g = a : ['b']
            b : ['c']
            c : ['a']
            d : ['b', 'b', 'e']
        sccs = [['d'], ['e'], ['a', 'c', 'b']]
    outout_t: {'d': ['abc', 'e']}
    '''
    vertex_scc_map = {}
    '''
        {
            'd': 0,
            'e': 1,
            'a': 2,
            'c': 2,
            'b': 2,
        }
    '''
    for i, scc in enumerate(sccs):
        for v in scc:
            vertex_scc_map[v] = i
    print(vertex_scc_map)
    
    cg = defaultdict(set)
    
    for u, vs in g.items():
        head_scc = vertex_scc_map[u]
        for v in vs:
            tail_scc = vertex_scc_map[v]
            if head_scc == tail_scc:
                continue
            cg[head_scc].add(tail_scc)
            
    return cg       

In [67]:
assert to_component_graph(g, sccs) == {0: {1,2}}

{'d': 0, 'e': 1, 'a': 2, 'c': 2, 'b': 2}
